### Tensorflow Estimator Bring your own Script

In this notebook we will go through and run a tensorflow model to classify the junctions as priority, signal and roundabout as seen in data prep.

The outline of this notebook is 

1. To prepare a training script (provided).

2. Use the AWS provided Tensorflow container and provide our script to it.

3. Run training.

4. Deploy model to end point.

5. Test using an image in couple of possible ways 

Upgrade Sagemaker so we can access the latest containers

In [ ]:
import datetime

sttime = datetime.datetime.now()

In [ ]:
!pip install -U sagemaker>=2.48

Let us also upgrade out version of Tensorflow to v2.4.1

In [ ]:
!pip install tensorflow==2.4.1

Lets make sure that our environment is using Tensorflow 2.4.1 otherwise we will need to restart the notebook kernel

In [ ]:
import tensorflow as tf

print(f"Tensorflow version {tf.__version__}")

if tf.__version__ != "2.4.1":
    print("This notebook kernel needs to be restarted!!!!")
    exit()

Lets start by importing some libraries that we will be using later

In [ ]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
# if you are using pytorch
# from sagemaker.pytorch import PyTorch


ON_SAGEMAKER_NOTEBOOK = True

sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = "[YOUR ROLE]"


A quick sanity check to make sure we are using the latest version of SageMaker

In [ ]:
sagemaker.__version__

#### Input params for model training 

In the cell below, replace **"your-unique-bucket-name"** with the name of bucket you created in the data-prep notebook

In [ ]:
bucket = "your-unique-bucket-name"
key = ""                            # Path from the bucket's root to the dataset


train_instance_type='ml.m5.12xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m5.4xlarge'     # The type of EC2 instance which will be used for deployment

'''
we can use the train and validation path as stated above 
or you can 
just rearrange data and use a single path like below
'''
training_data_uri="s3://{}".format(bucket)

### Tensorflow Estimator

Use AWS provided open source containers, these containers can be extended by starting with the image provided by AWS and the add additional installs in dockerfile

or you can use requirements.txt in source_dir to install additional libraries.

We setup the Tensorflow estimator job a job name, an entry point (which is our script **tfModelCode.py**), role, Tensorflow framework version, python version, instance count and type. <br>
Then we call the estimators fit method with the URI of the training dataset to kick off the training job.<br>
**Note: This cell will take approx 40-50 mins to complete**


In [ ]:
%%time
estimator_tf = TensorFlow(
  base_job_name='tensorflow-pssummit-traffic-class',
  entry_point="tfModelCode.py",             # Your entry script
  role=role,
  framework_version="2.4.1",               # TensorFlow's version
  py_version="py37",
  instance_count=1,  # "The number of GPUs instances to use"
  instance_type=train_instance_type,
)

print("Training ...")
estimator_tf.fit(training_data_uri)

#### Deploying a model
Once trained, deploying a model is a simple call. <br>
We specify two prarameters<br>
    **instance_type** - the type of the instance will be used to do inference<br>
    **initial_instance_count** - the initial number of instances that will be provisioned to do inference

In [ ]:
estimator_deployed=estimator_tf.deploy(instance_type='ml.m5.2xlarge', initial_instance_count=1)

Now that the estimator has been deployed to an endpoint, lets find out the endpoint name

In [ ]:
print(estimator_deployed.endpoint_name)

So to do predictions againast this endpoint, we are going to use Predictor. We provide it the endpoint name, the SageMaker session and the serializer (in our case a JSONSerializer)
Serializers implement methods for serializing data for an inference endpoint<br>
**NOTE** Replace **'your-endpoint-name'** with your endpoint name

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

endpoint_name = 'your-endpoint-name'

predictor=Predictor(endpoint_name=endpoint_name,
                    sagemaker_session=sagemaker_session, 
                    serializer=JSONSerializer())

Here we install some convenience libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import json

In [ ]:
vars(tf)

Now we will take one of our test images and apply some preprocessing to it

In [ ]:
file='../data/test/Roundabout/R2.png'
img = tf.keras.preprocessing.image.load_img(file, target_size=[250, 250])
plt.imshow(img)
plt.axis('off')
x = tf.keras.preprocessing.image.img_to_array(img)
x = tf.keras.applications.efficientnet.preprocess_input(
    x[tf.newaxis,...])


Now we send that processed data to our endpoint.

In [ ]:
predictor.predict(x)

As you can see the prediction has sent back a confidence score for each class. The second value in the list corresponds to the class label "Roundabout" which has the highest confidence score

### Using boto3 sagemaker_runtime client

So what if we want to make predictions against this endpoint outside of this notebook?<br>
We then leverage the boto3 library. <br>
**NOTE** Replace **'your-endpoint-name'** with your endpoint name

In [ ]:
import boto3
client=boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
            EndpointName='your-endpoint-name',
            ContentType='application/json',
            Body=json.dumps({'instances':x.tolist()}))

We can now view the JSON response. Again the second value in the list corresponds to the class label "Roundabout" which has the highest confidence score

In [ ]:
json.loads(response['Body'].read().decode("utf-8"))

## Batch Inference

We will start by creating a S3 URI to the model artifacts package generated from the training step

In [ ]:
model_data = f"{estimator_tf.output_path}{estimator_tf._current_job_name}/output/model.tar.gz"

Now we will create a local export folder so we can store our inference code in a code folder. We can also specify a requirements.txt for any package dependencies  

In [ ]:
! mkdir ./export

Now we copy and unpack the model artifacts file

In [ ]:
!aws s3 cp {model_data} ./export/

In [ ]:
!tar -xvzf ./export/model.tar.gz -C ./export/

In [ ]:
%cd export

We now delete any old model artifacts folders and move the unpacked model artifacts folder to the 1 folder

In [ ]:
! rm -r 1

In [ ]:
! mv tf000000001/1 .

In [ ]:
! rm -r code/.ipynb_checkpoints/

We now package up the code and 1 folder to create a new model.tar.gz file

In [ ]:
! tar -czvf model.tar.gz code 1

We copy the new model.tar.gz to your S3 bucket and setup our Tensorflow serving container

In [ ]:
import os
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
sm_role=sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
# See the following document for more on SageMaker Roles:
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html
role = sm_role

# Will be using the bucket variable defined at beginning of this notebook

prefix = 'tf_model'
s3_path = 's3://{}/{}'.format(bucket, prefix)

model_data = sagemaker_session.upload_data('model.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))
                                           
tensorflow_serving_model = TensorFlowModel(model_data=model_data,
                                 role=role,
                                 framework_version='2.4.1',
                                 sagemaker_session=sagemaker_session)

We then specify the output folder and run the transformer method to start the batch processing

In [ ]:
output_path = f's3://{bucket}/{prefix}/output'
tensorflow_serving_transformer = tensorflow_serving_model.transformer(
                                     instance_count=2,
                                     instance_type='ml.m5.4xlarge',
                                     max_concurrent_transforms=64,
                                     max_payload=1,
                                     output_path=output_path)

input_path = f's3://{bucket}/test'
tensorflow_serving_transformer.transform(input_path, content_type='application/x-image')

In [ ]:
output_path

We can look at the output file from the batch job. Each file is a prediction that corresponds to the input image file name 

In [ ]:
! aws s3 ls {output_path} --recursive | grep -v ".ipy"

In [ ]:
endtime = datetime.datetime.now()
str(endtime-sttime)

# Appendix and Utilities

### Attach to a training job that has been left to run 

If your kernel becomes disconnected and your training has already started, you can reattach to the training job.<br>
Simply look up the training job name and replace the **your-training-job-name** and then run the cell below. <br>
Once the training job is finished, you can continue the cells after the training cell

In [ ]:
import tensorflow as tf
import sagemaker
import boto3

sess = sagemaker.Session()

training_job_name = 'your-training-job-name'

estimator_tf = sagemaker.estimator.Estimator.attach(training_job_name=training_job_name, sagemaker_session=sess)

# Raj needs to review to see if we still want to keep the content below

### Sagemaker 2 update endpoint steps

import sagemaker

predictor = sagemaker.pytorch.model.PyTorchPredictor(existing_endpoint_name)
predictor.update_endpoint(initial_instance_count=1, instance_type="ml.p3.2xlarge", model_name= new_model_name)

### Ditributed Data Parallel - Code Modification

https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-modify-sdp-tf2.html

https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-use-api.html

```
import tensorflow as tf

**SageMaker data parallel: Import the library TF API **
import smdistributed.dataparallel.tensorflow as sdp

**SageMaker data parallel: Initialize the library**
sdp.init()

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    # SageMaker data parallel: Pin GPUs to a single library process
    tf.config.experimental.set_visible_devices(gpus[sdp.local_rank()], 'GPU')

**Prepare Dataset**
dataset = tf.data.Dataset.from_tensor_slices(...)

**Define Model**
mnist_model = tf.keras.Sequential(...)
loss = tf.losses.SparseCategoricalCrossentropy()

**SageMaker data parallel: Scale Learning Rate**
*LR for 8 node run : 0.000125*
*LR for single node run : 0.001*
opt = tf.optimizers.Adam(0.000125 * sdp.size())

@tf.function
def training_step(images, labels, first_batch):
    with tf.GradientTape() as tape:
        probs = mnist_model(images, training=True)
        loss_value = loss(labels, probs)

    **SageMaker data parallel: Wrap tf.GradientTape with the library's DistributedGradientTape**
    tape = sdp.DistributedGradientTape(tape)

    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    opt.apply_gradients(zip(grads, mnist_model.trainable_variables))

    if first_batch:
       **SageMaker data parallel: Broadcast model and optimizer variables**
       sdp.broadcast_variables(mnist_model.variables, root_rank=0)
       sdp.broadcast_variables(opt.variables(), root_rank=0)

    return loss_value

# SageMaker data parallel: Save checkpoints only from master node.
if sdp.rank() == 0:
    checkpoint.save(checkpoint_dir)
```

#### On estimator side
```
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    base_job_name = "training_job_name_prefix",
    entry_point="tf-train.py",
    role="SageMakerRole",
    framework_version="2.4.1",
    # You must set py_version to py36
    py_version="py37",
    # For training with multi node distributed training, set this count.
    # Example: 2,3,4,..8
    instance_count=2,
    # For training with p3dn instance use - ml.p3dn.24xlarge
    instance_type="ml.p3.16xlarge",
    # Training using smdistributed.dataparallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}}
)

tf_estimator.fit("s3://bucket/path/to/training/data")
```